# How much is your Fiat 500 worth?
## Skills: Python, Web scraping, Geopy, Regression

Download the Jupyter Notebook from [GitHub](https://nicomoriuala.github.io)

<a id='content' />

## Contents

1. [Introduction](#intro)
1. [Web Scraping](#scraping)
2. [Data Cleansing](#cleansing)
3. [Mapping](#mapping)
4. [A definir](#volatility)

<a id='intro' />

## Introduction
[(back to top)](#content)

In [1]:
#Importing packages
import pandas as pd
import numpy as np
import re
import unidecode

from tqdm.notebook import tqdm
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

<a id='scraping' />

## Web Scraping
[(back to top)](#content)

In [36]:
#Load webdriver
driver = webdriver.Chrome('/Users/nicolamori/Library/Application Support/binman_chromedriver/mac64/85.0.4183.38/chromedriver')

#Open page
driver.get('https://www.automobile.it/fiat-500-usate?anno_fino_a=2020&immatricolazione=2007')
time.sleep(4)

#Close cookies popup
driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
time.sleep(2)

In [37]:
#Empty list and df
temp = []
df_ = pd.DataFrame(columns=['Title', 'Province','Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission', 'Owners', 'Price'])

#Number of pages
npages = int(driver.find_element_by_xpath('//*[@id="__next"]/main/div[2]/div/div[4]/div[2]/div[3]/div[1]/button[4]').text)

for h in tqdm(range(npages), position = 0, desc='1st loop'):
    

    #Title and Province
    titles=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879"]')
    for title in titles: temp.append(title.text)
    
    del temp[1::3]
    temp = [temp[x:x+2] for x in range(0, len(temp),2)]
    
    #Dataframe and clear list
    part_1 = pd.DataFrame(temp, columns=['Title', 'Province'])  
    temp.clear()
    
    
    #Specs
    specs=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoTag font-base"]')
    for spec in specs: temp.append(spec.text)
    
    while 'Unico proprietario' in temp: temp.remove('Unico proprietario')
    while '2 Proprietari' in temp: temp.remove('2 Proprietari')
    while '3 Proprietari' in temp: temp.remove('3 Proprietari')
    while '4 Proprietari' in temp: temp.remove('4 Proprietari')
        
    size = len(temp) 
    idx_list = [idx for idx, val in enumerate(temp) if val == 'Usato'] 
    temp = [temp[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))] 
    del temp[0:0 + 1]
    
    #Dataframe and clear list
    part_2 = pd.DataFrame(temp, columns=['Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission'])
    temp.clear()
    
    
    #Price
    prices=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoPrice font-medium"]')
    for price in prices: temp.append(price.text)
        
    #Dataframe and clear list
    part_3 = pd.DataFrame(temp, columns=['Price'])
    temp.clear()
    
    
    #Combine parts and append to dataframe
    df = pd.concat([part_1.reset_index(drop=True), part_2.reset_index(drop=True), part_3.reset_index(drop=True)], axis=1)
    df_ = df_.append(df, ignore_index = True)
    
    
    #Switch to next page or close
    if h < max(range(npages)):
        driver.find_element_by_xpath('//*[@class="jsx-3190051910 font-base xxsmall primary-orange styled"]').click()
        time.sleep(3)
    else: driver.close()
    
    
#df_.to_csv('fiat500_prices.tsv', sep='\t', index=False)


CPU times: user 26.1 s, sys: 2.32 s, total: 28.4 s
Wall time: 20min 55s


In [20]:
#driver.close()